<img src="https://upload.wikimedia.org/wikipedia/commons/c/c7/HEIG-VD_Logo_96x29_RVB_ROUGE.png" alt="HEIG-VD Logo" width="250" /> 

# Cours TAL - Laboratoire 2
# Mise en œuvre et évaluation de *POS taggers* pour le français

**Objectif**

Appliquer des étiqueteurs morphosyntaxiques (POS taggers) disponibles dans NLTK et dans les outils Stanford NLP à des textes français, puis quantifier leurs performances.

**Instructions initiales**

* Télécharger l'archive `UD_French-GSD-withBlankLines.zip` fournie sur Cyberlearn.
* Placer les trois fichiers qu'elle contient dans le même dossier que le notebook.
* Ce sont des textes en français annotés avec les POS tags, provenant du projet ([Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD)), et légèrement modifiés.
  - le fichier `fr-ud-train.conllu3` est destiné à l'entraînement
  - le fichier `fr-ud-dev.conllu3` est destiné aux tests préliminaires et aux réglages des paramètres
  - le fichier `fr-ud-test.conllu3` est destiné à l'évaluation finale.

**Questions préliminaires**

* En inspectant les fichiers, veuillez indiquer le numéro de la colonne où se trouvent les mots, et celui de la colonne où se trouvent leur étiquettes morpho-syntaxiques (*POS tags*).
* Veuillez chercher sur le Web la liste des *POS tags* du projet Universal Dependencies, avec leurs définitions, et indiquer l'URL ci-dessous.


(Si la première colonne est la colonne numéro 1)

Colonne des mots: 2

Colonne des POS tags: 4

Lien des POS tags du projet Universal Dependencies: https://universaldependencies.org/u/pos/

* Veuillez déterminer et afficher le nombre de tokens de chacun des trois fichiers.

In [1]:
file_dev = "fr-ud-dev.conllu3"
file_train = "fr-ud-train.conllu3"
file_test = "fr-ud-test.conllu3"

files = [file_dev, file_train, file_test]

for file in files:
    with open(file, "r") as f:
        # On compte le nombre de lignes dans le fichier sans compter les lignes vides
        nb_lines = len([line for line in f if line.strip() != ""])
        print("Nombre de tokens dans le fichier {}: {}".format(file, nb_lines))

Nombre de tokens dans le fichier fr-ud-dev.conllu3: 36830
Nombre de tokens dans le fichier fr-ud-train.conllu3: 366371
Nombre de tokens dans le fichier fr-ud-test.conllu3: 10298


## Partie 1 : Évaluer le Stanford POS tagger avec les modèles fournis pour le français

L'Université de Stanford fournit un étiqueteur morpho-syntaxique (POS tagger) qui utilise l'apprentissage automatique (https://nlp.stanford.edu/software/tagger.html) appelé Maxent Tagger.  Le tagger et ses modèles multilingues peuvent être téléchargés à l'URL ci-dessus (archive ZIP suivant le lien *Download > full Stanford Tagger version 3.9.2*, 130 MB environ).  

Pour simplifier, on vous propose de télécharger séparément le programme Java [stanford-postagger.jar](https://drive.switch.ch/index.php/s/hMY6yO7lmoQJuS3) et le modèle français [french-ud.tagger](https://drive.switch.ch/index.php/s/4HSqKRTTTkCgPfB) fournis par l'enseignant (mot de passe = reference).  Enregistrez ces deux fichiers dans le même dossier que ce notebook.

Le Maxent Tagger est en Java, et peut être exécuté depuis ce notebook avec un appel Java en ligne de commande.  Pour exécuter une commande système depuis le notebook, ajouter '!' devant (par exemple `! dir` ou `! ls`).  Utilisez la [documentation du Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html), et plus précisément la section *Tagging and Testing from the command line*, pour comprendre comment l'invoquer.  Java doit être installé sur votre système, et si nécessaire, exécuter :
```python
import os
java_path = 'C:/Program Files (x86)/Java/jdk1.8.0_20/bin/java.exe'  # votre chemin de java.exe
os.environ['JAVA_HOME'] = java_path   # attention aux slash (pas backslash sous Windows)
```
*Note* : il est également possible d'appeler ce tagger avec des commandes NLTK grâce au module [nltk.tag.stanford](https://www.nltk.org/_modules/nltk/tag/stanford.html) mais la gestion des *paths* entre Java, les classes et les modèles peut être compliquée.

In [2]:
from nltk.tag.stanford import StanfordPOSTagger

stanford_dir = './'
modelfile = stanford_dir + 'french-ud.tagger'
jarfile = stanford_dir + 'stanford-postagger.jar'

st = StanfordPOSTagger(model_filename=modelfile,
                       path_to_jar=jarfile, verbose=False)

**Question**

Appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-dev.conllu3` et demandez à Maxent Tagger de mesurer la qualité par comparaison à une l'annotation de référence fournie dans le fichier. Quels sont les scores obtenus ?  Quel est le nombre le plus important?  Indiquez ces réponses en commentaires du code.

In [3]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model 'french-ud.tagger' -testFile 'format=TSV,wordColumn=1,tagColumn=3,fr-ud-dev.conllu3' -verboseResults false

# Phrases: 1478
#   Correctes:  144  (9.742896%)
#   Fausses:    1334 (90.257104%)

# Tags: 36830
#   Corrects:   32360 (87.863155%)
#   Faux:       4470  (12.136845%)

# Mots inconnus: 3049
#   Corrects:   2232 (73.204329%)
#   Faux:       817  (26.795671%)

 
# Le nombre le plus important est le nombre de tags.

# Nous observons que le tagger s'en sort bien avec les tags et mêmes les mots inconnus mais qu'il a plus de mal avec les phrases. Ceci s'explique assez facilement car une phrase est considérée comme fausse si au moins un mot est mal taggé. Ce qui est le cas pour la plupart des phrases.


Loading default properties from tagger french-ud.tagger
Loading POS tagger from french-ud.tagger ... done [0.1 sec].
Tagged 36830 words at 46327,04 words per second.
Model french-ud.tagger has xSize=304855, ySize=18, and numFeatures=104853.
Results on 1478 sentences and 36830 words, of which 3049 were unknown.
Total sentences right: 144 (9,742896%); wrong: 1334 (90,257104%).
Total tags right: 32360 (87,863155%); wrong: 4470 (12,136845%).
Unknown words right: 2232 (73,204329%); wrong: 817 (26,795671%).


De même, appliquez le Maxent Tagger pour étiqueter le fichier `fr-ud-test.conllu3` et indiquez la précision du tagger en commentaires du code (#).

In [23]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger -model 'french-ud.tagger' -testFile 'format=TSV,wordColumn=1,tagColumn=3,fr-ud-test.conllu3' -verboseResults false

# Précision du tagger:
#
#   Phrases: ~12.98%
#   Tokens: ~87.01%
#   Mots inconnus: ~69.87%

Loading default properties from tagger french-ud.tagger
Loading POS tagger from french-ud.tagger ... done [0.1 sec].
Tagged 10298 words at 37860,29 words per second.
Model french-ud.tagger has xSize=304855, ySize=18, and numFeatures=104853.
Results on 416 sentences and 10298 words, of which 697 were unknown.
Total sentences right: 54 (12,980769%); wrong: 362 (87,019231%).
Total tags right: 8960 (87,007186%); wrong: 1338 (12,992814%).
Unknown words right: 487 (69,870875%); wrong: 210 (30,129125%).


**Question subsidiare** : combien de phrases et de mots le tagger trouve-t-il dans les fichiers `fr-ud-dev.conllu3` et `fr-ud-test.conllu3` ?  Comparez avec votre propre estimation du nombre de mots.

### Pour le fichier de dev:

Le tagger trouve 36830 mots et 1478 phrases. 

Nous avions estimé 36830 mots ce qui est exact. 

### Pour le fichier de test:

Le tagger trouve 10298 mots et 416 phrases. 

Nous avions estimé 10298 mots ce qui est exact. 

Nous avions donc bien estimé le nombre de mots dans les fichiers en comptant le nombre de lignes non vides.

## Partie 2 : Entraîner le Stanford POS tagger pour obtenir de nouveaux modèles

Le but de cette partie est d'entraîner le Maxent Tagger sur les données UD en français (`fr-ud-train.conllu3`), puis de comparer le modèle obtenu avec les modèles fournis par Stanford pour le français, testés dans la partie 1A.  

Suivre la [documentation de Maxent Tagger](https://nlp.stanford.edu/nlp/javadoc/javanlp/edu/stanford/nlp/tagger/maxent/MaxentTagger.html) pour l'entraîner sur le fichier `fr-ud-train.conllu3` et le tester sur `fr-ud-test.conllu3`.  Regardez la section *Training from the command line*. 

La configuration du système pour effectuer l'entraînement est donnée dans un fichier texte, qui peut être produit en suivant la documentation (option `-genprops` pour obtenir un template qui sera édité), soit en s'inspirant du fichier [french-ud.tagger.props](https://drive.switch.ch/index.php/s/gHlam9S74HG2Q4X) accompagnant le modèle `french-ud.tagger` que vous avez utilisé ci-dessus.  Pensez à donner un nouveau nom à votre fichier modèle.

**Questions**

* Créez un fichier `myFrench-ud.tagger.props` qui aboutit à un bon entraînement.  Vous pourrez expérimenter plusieurs fois et proposer le meilleur fichier.  Citez dans le notebook les paramètres sur lesquels vous avez agi.

* Lancez l'entraînement sur le fichier `fr-ud-train.conllu3` (s'il ne tient pas en mémoire, utilisez seulement `fr-ud-dev.conllu3`). Pendant l’entraînement (> 10 minutes, 500 itérations), regardez la suite du travail.

* Évaluez votre modèle comme ci-dessus (sur `dev` et sur `test`).  Quel modèle est meilleur, le vôtre ou celui fourni par Stanford ?  Formulez une hypothèse expliquant ce résultat. 

In [30]:
# Créer un fichier de propriétés pour le tagger vierge (finalement non utilisé)
# !java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger \
#   -genprops > myFrench-ud.tagger.props

Nous avons modifié les paramètres suivantes dans le fichier `myFrench-ud.tagger.props`:
* `iterations` : 500 au lieu de 100
* `rareWordMinFeatureThresh` : 3 au lieu de 10 (Discard les rareswords qui apparaisent moins de 3 fois)
* `rareWordThresh` : 10 au lieu de 5 (Condidère qu'un mot est rare s'il apparait moins de 10 fois)

Nous avons donc augmenté légèrement le nombre de mots qui seront considérés comme rares mais nous avons réduit le minimum de fois qu'un mot doit apparaître pour être discard. Cela permet de garder environ le même nombre de mots dans le modèle. Nous avons également augmenté le nombre d'itérations pour que le modèle soit plus précis.

In [35]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger \
  -props 'myFrench-ud.tagger.props' \
  -trainFile 'format=TSV,wordColumn=1,tagColumn=3,fr-ud-dev.conllu3' \
  -verboseResults false \
  -iterations 500

## tagger training invoked at Mon Mar 13 10:52:58 CET 2023 with arguments:
                   model = myFrench-ud.tagger
                    arch = left3words,naacl2003unknowns,unicodeshapes(-1,1)
            wordFunction = 
               trainFile = format=TSV,wordColumn=1,tagColumn=3,fr-ud-dev.conllu3
         closedClassTags = 
 closedClassTagThreshold = 40
 curWordMinFeatureThresh = 2
                   debug = false
             debugPrefix = 
            tagSeparator = _
                encoding = utf-8
              iterations = 500
                    lang = french
    learnClosedClassTags = false
        minFeatureThresh = 2
           openClassTags = 
rareWordMinFeatureThresh = 3
          rareWordThresh = 7
                  search = qn
                    sgml = false
            sigmaSquared = 0.0
                   regL1 = 0.75
               tagInside = 
                tokenize = true
        tokenizerFactory = 
        tokenizerOptions = asciiQuotes
                 v

In [39]:
!java -cp stanford-postagger.jar edu.stanford.nlp.tagger.maxent.MaxentTagger \
  -model 'myFrench-ud.tagger' \
  -testFile 'format=TSV,wordColumn=1,tagColumn=3,fr-ud-test.conllu3' \
  -verboseResults false

# Pour comparer, nous remettons les précisions du tagger de Stanford:
#   Phrases: ~12.98%
#   Tokens: ~87.01%
#   Mots inconnus: ~69.87%

# Results on 416 sentences and 10298 words, of which 1687 were unknown.
# Total sentences right: 126 (30,288462%); wrong: 290 (69,711538%).
# Total tags right: 9579 (93,018062%); wrong: 719 (6,981938%).
# Unknown words right: 1370 (81,209247%); wrong: 317 (18,790753%).

Loading default properties from tagger myFrench-ud.tagger
Loading POS tagger from myFrench-ud.tagger ... done [0.0 sec].
Tagged 10298 words at 26070,89 words per second.
Model myFrench-ud.tagger has xSize=34166, ySize=19, and numFeatures=28553.
Results on 416 sentences and 10298 words, of which 1687 were unknown.
Total sentences right: 126 (30,288462%); wrong: 290 (69,711538%).
Total tags right: 9579 (93,018062%); wrong: 719 (6,981938%).
Unknown words right: 1370 (81,209247%); wrong: 317 (18,790753%).


### Quel modèle est meilleur, le vôtre ou celui fourni par Stanford ? Formulez une hypothèse expliquant ce résultat.

#### Précisions du tagger de Stanford sur le fichier fr-ud-test.conllu3:
* Phrases: ~12.98%
* Tokens: ~87.01%
* Mots inconnus: ~69.87%

#### Précisions du tagger de notre modèle sur le fichier fr-ud-test.conllu3:
TODO mettre les valeurs du run de vali
* Phrases: ~39,90%
* Tokens: 95,39%
* Mots inconnus: ~82,70%

Notre modèle a donc des valeurs bie meilleures que celui de Stanford. En effet, notre modèle atteint les 95% de précision sur les tokens. Mais la partie la plus impressionnante est le nombre de phrases correctement taggées. En effet, notre modèle tagge 39,90% des phrases correctement alors que Stanford n'en tagge que 12,98%. C'est environ 3 fois plus.

Nous pouvons expliquer ce résultat sur le fait que nous réalisons un nombre d'itérations plus important que celui fourni par Stanford (500 vs 100). De plus, les modifications sur les rares words doivent avoir un impact bénéfique sur nos résultats. 

Nous pouvons également supposer que le modèle fournit par stanford est prévu pour des textes en anglais et marche donc moins bien sur des données en français.

## Partie 3 : entraîner un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger *Averaged Perceptron* de NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus.  

Notez que pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK, comme expliqué au [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html) : on peut écrire `nltk.pos_tag(sentence)` où *sentence* est une phrase tokenisée. L'étiquetage morpho-syntaxique produira des paires ('mot', 'TAG').

**Première étape**

Importer les textes annotés `fr-ud-XXXX.conllu3` grâce à des objets `ConllCorpusReader`.  Consultez le mode d'emploi de cette classe directement dans [son code source](https://www.nltk.org/_modules/nltk/corpus/reader/conll.html#ConllCorpusReader), pour déterminer comment lire un fichier en créant un objet `ConllCorpusReader`.  Chargez les trois fichiers, dans trois objets appelés `train_corpus`, `dev_corpus` et `test_corpus`.

In [3]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [4]:
def get_corpus(file):
    return ConllCorpusReader(
        ".", 
        file, 
        ('ignore', 'words', 'ignore', 'pos'),
        separator="\t"
    )

train_corpus = get_corpus("fr-ud-train.conllu3")
dev_corpus = get_corpus("fr-ud-dev.conllu3")
test_corpus = get_corpus("fr-ud-test.conllu3")

print(train_corpus.tagged_words())
print(dev_corpus.tagged_words())
print(test_corpus.tagged_words())

[('Les', 'DET'), ('commotions', 'NOUN'), ...]
[('Aviator', 'PROPN'), (',', 'PUNCT'), ('un', 'DET'), ...]
[('Je', 'PRON'), ('sens', 'VERB'), ("qu'", 'SCONJ'), ...]


Affichez le nombre de phrases et le nombre de mots de chaque corpus chargé. Ces chiffres sont-ils identiques à ceux obtenus pour `dev` et pour `test` à la fin de la Partie 1 ? On peut obtenir les listes de mots étiquetés avec `tagged_words()` et les listes de phrases avec mots étiquetés avec `tagged_sents()`.

In [5]:
all_corpus = [("Train", train_corpus), ("Dev", dev_corpus), ("Test", test_corpus)]

for corpus in all_corpus:
    print(f"{corpus[0]} -------------")
    print(f"Nombre de mots: {len(corpus[1].tagged_words())}")
    print(f"Nombre de phrases: {len(corpus[1].tagged_sents())}\n")

Train -------------
Nombre de mots: 366371
Nombre de phrases: 14554

Dev -------------
Nombre de mots: 36830
Nombre de phrases: 1478

Test -------------
Nombre de mots: 10298
Nombre de phrases: 416



### Comparaison avec les résultats de la fin de la partie 1

**Dev:** Le tagger trouve 36830 mots et 1478 phrases. 

**Test:** Le tagger trouve 10298 mots et 416 phrases. 

Les résultats sont donc les mêmes pour les deux corpus avec la partie 1.


Affichez la 17e phrase du corpus de développement (avec les étiquettes POS), et les mots 1001 à 1050 du corpus de test (aussi avec leurs POS tags).

In [6]:
print("17e phrase")
print(f"{dev_corpus.tagged_sents()[16]}\n")

print("Mots 1001 -> 1050")
print(test_corpus.tagged_words()[1000:1049])

17e phrase
[('Comprenant', 'VERB'), ('six', 'NUM'), ('sommets', 'NOUN'), ('dont', 'PRON'), ('un', 'DET'), ('point', 'NOUN'), ('culminant', 'VERB'), ('à', 'ADP'), ('2 001', 'NUM'), ('mètres', 'NOUN'), ('et', 'CCONJ'), ('une', 'DET'), ('arrivée', 'NOUN'), ('en', 'ADP'), ('altitude', 'NOUN'), (',', 'PUNCT'), ("c'", 'PRON'), ('est', 'AUX'), ('une', 'DET'), ('étape', 'NOUN'), ('typique', 'ADJ'), ('de', 'ADP'), ('montagne', 'NOUN'), ('.', 'PUNCT')]

Mots 1001 -> 1050
[('dans', 'ADP'), ('la', 'DET'), ('raison', 'NOUN'), ('politique', 'ADJ'), ('...', 'PUNCT'), ('Mais', 'CCONJ'), ('la', 'DET'), ('réalité', 'NOUN'), ('est', 'VERB'), ('que', 'SCONJ'), ('la', 'DET'), ('Mauritanie', 'PROPN'), ("n'", 'ADV'), ('est', 'AUX'), ('pas', 'ADV'), ('le', 'DET'), ('Maroc', 'PROPN'), ('ou', 'CCONJ'), ("l'", 'DET'), ('Algérie', 'PROPN'), ('.', 'PUNCT'), ('En', 'ADP'), ('Arabie', 'PROPN'), (',', 'PUNCT'), ('on', 'PRON'), ('a', 'VERB'), ("l'", 'DET'), ('impression', 'NOUN'), ('que', 'SCONJ'), ('le', 'DET'), ('fo

**Seconde étape**

Vous allez maintenant entraîner (sur le corpus `train`) le POS tagger appelé *Averaged Perceptron* fourni par NLTK mais [implémenté par Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python).

Dans le [package de NLTK avec des taggers](http://www.nltk.org/api/nltk.tag.html), considérez le module `nltk.tag.perceptron`, pour lequel NLTK explique de façon précise l'entraînement (voir *train the model*) et le test.  Vous allez mettre en oeuvre ces étapes pour entraîner le tagger.  Notez que le modèle est enregistré dans un fichier qui doit finir par `.pickle`, et qui est écrasé à chaque entraînement si vous ne changez pas de nom.  Un modèle peut être également chargé dans un tagger.

In [7]:
# import os # si nécessaire
# import nltk # si nécessaire
# nltk.download('averaged_perceptron_tagger') # si nécessaire
from nltk.tag.perceptron import PerceptronTagger

In [8]:
ptagger = PerceptronTagger(load=False)

Entraînez ici le tagger sur les données d'entraînement, avec les meilleurs paramètres possibles.

In [15]:
import time
start_time = time.time()

print("Début de l'entraînement...")
ptagger.train(
    train_corpus.tagged_sents(),
    save_loc="training_model.pickle",
    nr_iter=10
)
print(f"Temps d'entraînement: {round((time.time() - start_time), 4)} secondes")

Début de l'entraînement...
Temps d'entraînement: 48.3787 secondes


Combien de temps prend l'entraînement ?  Quelle est la taille du fichier modèle résultant ?

L'entraînement prend environ **48.4 secondes** pour 10 itérations.

La taille du fichier modèle est de **8.4 Mo**.

Évaluez le tagger, d'abord sur les données `dev` puis sur les données `test`.

In [21]:
print(ptagger.accuracy(dev_corpus.tagged_sents()))
print(ptagger.accuracy(test_corpus.tagged_sents()))

0.9671191963073581
0.9597980190328219


Veuillez remplir le tableau suivant avec la synthèse des résultats.

| Corpus | MaxEnt | MaxEnt   | Avg Perceptron | 
|--------|--------|----------|---------------|
| -      | fourni | entraîné | entraîné |
| dev    |   0.8786   |   ..     |  0.9671  |
| test   |   0.8701   |   ..     |  0.9598  |

Comment se comparent les deux POS taggers sur le français ?  Écrivez vos conclusions dans cette cellule.

TODO

Avantager partie 3: temps d'exécution  
Désavantage partie 3: taille du fichier modèle (8.4 Mo vs 1.1 Mo)

## Fin du laboratoire 2  

Merci de nettoyer votre feuille, exécuter une dernière fois toutes les instructions, sauvegarder le résultat, et le rendre via Cyberlearn.